# <div style="text-align: right;font-family:Times New Roman">Приложение Б-3</div>
# <div style="text-align: center;font-family:Times New Roman">Сравнение моделей</div>

# Model comparison
# Сравнение моделей
---
In this notebook different models - logistic regression, SVM, random forest, gradient boosting, dense neural network and U-Net - are applied to model the vegetation of 1/20 of a territory and their performances are compared

В данном блокноте разные модели - логистическая регрессия, метод опорных векторов, случайный лес, градиентный бустинг, полносвязная нейронная сеть и U-Net - применяются для моделирования растительности на 1/20 части территории и сравнивается их качество

### Importing dependencies
### Импорт необходимых библиотек

In [1]:
import os
import random
import re
import numpy as np
import matplotlib
from glob import glob
import rasterio as rio
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches, colors
import pandas as pd
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score, f1_score
from osgeo import ogr, osr
import xgboost as xgb

import fiona
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterio.merge
import rasterio.fill
from rasterio.io import MemoryFile
from rasterio.enums import Resampling
from rasterio.windows import Window
from rasterio.transform import Affine
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import earthpy.mask as em

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras import optimizers

C:\ProgramData\Anaconda3\envs\geo\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### Data preparation
### Подготовка данных

In [2]:
#loading x data / загрузка предикторов
path = r'F:\Work\CorrProizv\final_noadj\\'
files = glob(path+'*.tif')
files

['F:\\Work\\CorrProizv\\final_noadj\\B1.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B11.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B12.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B2.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B3.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B4.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B5.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B6.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B7.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B8.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B8A.tif',
 'F:\\Work\\CorrProizv\\final_noadj\\B9.tif']

In [7]:
x_train = []
for band in files:
    with rio.open(band) as bnd:
        rstr = bnd.read(1)
        x_train.append(rstr[img > 1])
    print(band)

F:\Work\CorrProizv\final_noadj\B1.tif
F:\Work\CorrProizv\final_noadj\B11.tif
F:\Work\CorrProizv\final_noadj\B12.tif
F:\Work\CorrProizv\final_noadj\B2.tif
F:\Work\CorrProizv\final_noadj\B3.tif
F:\Work\CorrProizv\final_noadj\B4.tif
F:\Work\CorrProizv\final_noadj\B5.tif
F:\Work\CorrProizv\final_noadj\B6.tif
F:\Work\CorrProizv\final_noadj\B7.tif
F:\Work\CorrProizv\final_noadj\B8.tif
F:\Work\CorrProizv\final_noadj\B8A.tif
F:\Work\CorrProizv\final_noadj\B9.tif


In [8]:
x_train

[array([103, 116, 125, ..., 225, 226, 244], dtype=uint16),
 array([ 276,  306,  347, ..., 1104, 1099, 1430], dtype=uint16),
 array([131, 146, 165, ..., 555, 566, 759], dtype=uint16),
 array([ 73,  85, 102, ..., 246, 252, 312], dtype=uint16),
 array([181, 177, 199, ..., 359, 419, 447], dtype=uint16),
 array([ 98, 109, 123, ..., 211, 222, 293], dtype=uint16),
 array([259, 274, 258, ..., 569, 608, 741], dtype=uint16),
 array([ 642,  688,  659, ..., 1720, 1647, 1965], dtype=uint16),
 array([ 697,  755,  766, ..., 2300, 2179, 2565], dtype=uint16),
 array([ 631,  679,  680, ..., 2040, 1990, 2554], dtype=uint16),
 array([ 688,  736,  738, ..., 2495, 2378, 2871], dtype=uint16),
 array([ 540,  594,  619, ..., 2260, 2257, 2773], dtype=uint16)]

In [9]:
#converting to pandas dataframe / конвертация в датафрейм pandas
x_train = pd.DataFrame(data = {'1': x_train[0], '2': x_train[1], '3': x_train[2], '4': x_train[3], '5': x_train[4], 
                               '6': x_train[5], '7': x_train[6], '8': x_train[7], '9': x_train[8], '10': x_train[9],
                               '11': x_train[10], '12': x_train[11]})
x_train

,1,2,3,4,5,6,7,8,9,10,11,12
0,103,276,131,73,181,98,259,642,697,631,688,540
1,116,306,146,85,177,109,274,688,755,679,736,594
2,125,347,165,102,199,123,258,659,766,680,738,619
3,135,350,179,122,205,128,248,627,727,685,709,618
4,157,373,184,189,264,143,302,681,708,854,667,665
...,...,...,...,...,...,...,...,...,...,...,...,...
95096420,210,1125,547,239,394,193,641,1762,2278,2224,2593,2409
95096421,188,972,451,204,389,188,561,1502,1891,1882,2180,2049
95096422,225,1104,555,246,359,211,569,1720,2300,2040,2495,2260
95096423,226,1099,566,252,419,222,608,1647,2179,1990,2378,2257


In [3]:
#loading y data / загрузка обучающих данных
y = r'E:\Work\CorrProizv\y_1.tif'

In [4]:
with rio.open(y) as bnd:
    img = bnd.read(1)

In [5]:
y_train = img[img > 1]

In [6]:
y_train

array([4, 4, 4, ..., 4, 3, 3], dtype=uint16)

In [10]:
#converting to pandas series / конвертация в серию pandas
y_train = pd.Series(data=y_train)
y_train

0           4
1           4
2           4
3           4
4           4
           ..
95096420    3
95096421    3
95096422    4
95096423    3
95096424    3
Length: 95096425, dtype: uint16

In [11]:
img = None
rstr = None

### Logit / Логистическая регрессия

In [32]:
logit = LogisticRegression()

In [33]:
logit.fit(x_train, y_train)

C:\ProgramData\Anaconda3\envs\geo\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [34]:
x_pred = logit.predict(x_train)

In [35]:
accuracy_score(y_train, x_pred)

0.6816483795263597

In [36]:
mean_squared_error(y_train, x_pred)

0.4737200793825846

In [37]:
roc_auc_score(tf.keras.utils.to_categorical(y_train - 2, num_classes=3), tf.keras.utils.to_categorical(x_pred - 2, num_classes=3))

0.645516229675525

### SVM (Support Vector Machine) / Метод опорных векторов

In [12]:
svm = LinearSVC(max_iter = 10)

In [ ]:
svm.fit(x_train, y_train)

In [ ]:
x_pred = svm.predict(x_train)

In [3]:
accuracy_score(y_train, x_pred)

0.801137829572957


In [4]:
mean_squared_error(y_train, x_pred)

0.316284675896952


In [5]:
roc_auc_score(tf.keras.utils.to_categorical(y_train - 2, num_classes=3), tf.keras.utils.to_categorical(x_pred - 2, num_classes=3))

0.781659738842042


### Random forest / Случайный лес

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf.fit(x_train, y_train)

In [ ]:
x_pred = rf.predict(x_train)

In [6]:
accuracy_score(y_train, x_pred)

0.82244932943182


In [7]:
mean_squared_error(y_train, x_pred)

0.32014956936104


In [8]:
roc_auc_score(tf.keras.utils.to_categorical(y_train - 2, num_classes=3), tf.keras.utils.to_categorical(x_pred - 2, num_classes=3))

0.796430568324985


### XGboost / Градиентный бустинг

In [44]:
xgbc = xgb.XGBClassifier()

In [17]:
xgbc.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:49:07] WARNING: D:\bld\xgboost-split_1631904903843\work\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='approx', validate_parameters=1, verbosity=None)

In [19]:
#saving model / сохранение модели
pickle.dump(xgbc, open("xgbc.pickle.dat", "wb"))

In [20]:
xgbc.save_model('xgbc1.json')

In [22]:
x_pred = xgbc.predict(x_train)

In [64]:
accuracy_score(y_train, x_pred)

0.8466998988485912

In [24]:
mean_squared_error(y_train, x_pred)

0.27254565378138085

In [76]:
f1_score(y_train, x_pred, average='micro')

0.8466998988485912

In [68]:
roc_auc_score(keras.utils.to_categorical(y_train - 2, num_classes=3), keras.utils.to_categorical(x_pred - 2, num_classes=3))

0.8216102231842349

### Dense neural network (perceptron) / Полносвязная нейронная сеть (перцептрон)

In [3]:
from tensorflow import keras

In [18]:
#converting y data to keras categorical format / конвертация обучающих данных в категориальный формат keras
y_train = y_train - 2

In [19]:
y_train

array([2, 2, 2, ..., 2, 1, 1], dtype=uint16)

In [20]:
y_train = keras.utils.to_categorical(
    y_train, num_classes=3
)

In [23]:
x_train

,1,2,3,4,5,6,7,8,9,10,11,12
0,131,1294,610,165,393,211,788,2138,2382,2480,2532,2579
1,121,1385,645,139,459,275,866,2322,2588,2724,2705,2788
2,122,1427,669,152,441,259,865,2356,2609,2784,2881,2891
3,138,1417,668,173,466,266,880,2297,2535,2670,2843,2835
4,146,1435,671,176,450,283,888,2114,2433,2592,2718,2737
...,...,...,...,...,...,...,...,...,...,...,...,...
95094199,210,1125,547,239,394,193,641,1762,2278,2224,2593,2409
95094200,188,972,451,204,389,188,561,1502,1891,1882,2180,2049
95094201,225,1104,555,246,359,211,569,1720,2300,2040,2495,2260
95094202,226,1099,566,252,419,222,608,1647,2179,1990,2378,2257


In [24]:
#converting x data to numpy array / конвертация предикторов в массив numpy
x_train = x_train.to_numpy()

In [32]:
x_train

array([[ 131, 1294,  610, ..., 2480, 2532, 2579],
       [ 121, 1385,  645, ..., 2724, 2705, 2788],
       [ 122, 1427,  669, ..., 2784, 2881, 2891],
       ...,
       [ 225, 1104,  555, ..., 2040, 2495, 2260],
       [ 226, 1099,  566, ..., 1990, 2378, 2257],
       [ 244, 1430,  759, ..., 2554, 2871, 2773]], dtype=uint16)

In [26]:
#random shuffling data and splitting to train and test
#случайное перемешивание данных и разбиение на тренировочную и тестовую выборки
samples = list(range(len(y_train)))

In [28]:
np.random.shuffle(samples)
samples

[66921040,
 42159898,
 88408867,
 58351929,
 59967678,
 19389264,
 30268879,
 60247734,
 63833411,
 42109713,
 85891440,
 13579409,
 89382946,
 39677134,
 1911718,
 72143436,
 5273515,
 32204433,
 29138745,
 93178616,
 46541160,
 3331142,
 10382305,
 21844564,
 30761233,
 91354915,
 7191236,
 64773929,
 68655205,
 26555416,
 70442730,
 30281104,
 86055393,
 90599200,
 38514638,
 31199386,
 6090882,
 58474481,
 18018011,
 43571650,
 52226241,
 21943262,
 11981878,
 53002301,
 8534974,
 27380906,
 66475979,
 49247827,
 14698339,
 4211532,
 79430582,
 24506282,
 36031035,
 69603938,
 58604924,
 36656070,
 55421508,
 47892969,
 60954128,
 94382468,
 16866757,
 30900210,
 42660982,
 28787134,
 20672543,
 62948977,
 1991493,
 26104087,
 7526330,
 70326687,
 82547273,
 18895527,
 80915552,
 32590386,
 21964004,
 4408938,
 898806,
 53374080,
 85562627,
 42287126,
 39529214,
 46071027,
 87375268,
 15300331,
 64172053,
 34093437,
 47248801,
 28352474,
 24889237,
 75196018,
 36194259,
 75598079,


In [29]:
len(samples)

95094204

In [30]:
train = samples[:80000000]
test = samples[80000000:]

In [31]:
x_test = x_train[test]
y_test = y_train[test]
x_test

array([[ 430, 2806, 1586, ..., 3526, 3817, 3842],
       [ 251,  245,   96, ...,  405,  260,  607],
       [ 312, 1371,  577, ..., 3164, 3293, 3304],
       ...,
       [ 279, 2435, 1110, ..., 5440, 5698, 5642],
       [ 153, 1343,  671, ..., 3752, 3858, 3893],
       [ 353,   84,   71, ...,  307,  279,  128]], dtype=uint16)

In [34]:
x_tr = x_train[train]
y_tr = y_train[train]
len(y_tr)

80000000

In [35]:
x_tr.shape

(80000000, 12)

In [36]:
len(x_test)

15094204

In [39]:
y_test

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [4]:
#creating model with 4 dense layers / создание модели с 4 полносвязными слоями
model = keras.models.Sequential()
model.add(keras.layers.Dense(64, activation='relu', input_shape=(12,)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ['accuracy', 'mse'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                832       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 51        
Total params: 3,491
Trainable params: 3,491
Non-trainable params: 0
_________________________________________________________________


In [41]:
#training / обучение
history = model.fit(x_tr, y_tr, epochs = 5, batch_size = 512, validation_data = (x_test, y_test))

Epoch 1/5
156250/156250 [==============================] - 968s 6ms/step - loss: 0.5092 - accuracy: 0.8223 - mse: 0.0869 - val_loss: 0.4469 - val_accuracy: 0.8385 - val_mse: 0.0796
Epoch 2/5
156250/156250 [==============================] - 958s 6ms/step - loss: 0.4629 - accuracy: 0.8326 - mse: 0.0822 - val_loss: 0.4554 - val_accuracy: 0.8390 - val_mse: 0.0805
Epoch 3/5
156250/156250 [==============================] - 964s 6ms/step - loss: 0.4643 - accuracy: 0.8327 - mse: 0.0819 - val_loss: 0.4489 - val_accuracy: 0.8356 - val_mse: 0.0798
Epoch 4/5
156250/156250 [==============================] - 963s 6ms/step - loss: 0.4540 - accuracy: 0.8350 - mse: 0.0808 - val_loss: 0.4544 - val_accuracy: 0.8378 - val_mse: 0.0801
Epoch 5/5
156250/156250 [==============================] - 950s 6ms/step - loss: 0.4516 - accuracy: 0.8368 - mse: 0.0803 - val_loss: 0.4517 - val_accuracy: 0.8367 - val_mse: 0.0806


In [79]:
pred_perc = model.predict(x_test)

In [82]:
np.argmax(pred_perc,axis=1)

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [84]:
f1_score(np.argmax(y_test,axis=1), np.argmax(pred_perc,axis=1), average='micro')

0.8367407781158914

In [69]:
roc_auc_score(y_test, model.predict(x_test))

0.9216162440301868

### U-NET

In [10]:
#importing data / импорт даннных
path = r'D:\Work\CorrProizv\\'
files = glob(path+'[0-9].tif') + glob(path+'[0-9][0-9].tif')
files

['D:\\Work\\CorrProizv\\1.tif',
 'D:\\Work\\CorrProizv\\2.tif',
 'D:\\Work\\CorrProizv\\3.tif',
 'D:\\Work\\CorrProizv\\4.tif',
 'D:\\Work\\CorrProizv\\5.tif',
 'D:\\Work\\CorrProizv\\6.tif',
 'D:\\Work\\CorrProizv\\7.tif',
 'D:\\Work\\CorrProizv\\8.tif',
 'D:\\Work\\CorrProizv\\9.tif',
 'D:\\Work\\CorrProizv\\10.tif',
 'D:\\Work\\CorrProizv\\11.tif',
 'D:\\Work\\CorrProizv\\12.tif']

In [2]:
y = r'D:\Work\CorrProizv\y.tif'

In [3]:
x_train_conv = []

In [6]:
for i in range(len(files)):
    with rio.open(files[i]) as bnd:
        x_train_conv.append(bnd)
        print(files[i])

F:\Work\CorrProizv\final_noadj\B1.tif
F:\Work\CorrProizv\final_noadj\B11.tif
F:\Work\CorrProizv\final_noadj\B12.tif
F:\Work\CorrProizv\final_noadj\B2.tif
F:\Work\CorrProizv\final_noadj\B3.tif
F:\Work\CorrProizv\final_noadj\B4.tif
F:\Work\CorrProizv\final_noadj\B5.tif
F:\Work\CorrProizv\final_noadj\B6.tif
F:\Work\CorrProizv\final_noadj\B7.tif
F:\Work\CorrProizv\final_noadj\B8.tif
F:\Work\CorrProizv\final_noadj\B8A.tif
F:\Work\CorrProizv\final_noadj\B9.tif
F:\Work\CorrProizv\DEM.tif
F:\Work\CorrProizv\Landcover.tif
F:\Work\CorrProizv\Hansen_mosaic.tif


In [ ]:
x_train_conv1 = np.stack(x_train_conv, axis = -1)

In [97]:
x_train_conv1 = np.transpose(x_train_conv1, axes=[3, 0, 1, 2])

In [11]:
x_train_conv1 = x_train_conv1[0, :, :, :]

In [12]:
x_train_conv1.shape

(30613, 38128, 12)

In [3]:
with rio.open(y) as bnd:
    y_train_conv = bnd.read().astype('uint16')
    final_trans = bnd.transform
    #y_train_conv, final_trans = rio.mask.mask(bnd, shape,crop=True, filled=True)

In [7]:
#generating tiles / генерация тайлов
tiles = []
x1 = 0
y1 = 0
x2 = 128
y2 = 128
for i in range (0,632,1):
    for i in range(0,1089,1):
        tiles.append((x1,y1,x2,y2))
        y1 +=128
        y2 +=128
    y1=0
    y2=128
    x1+=128
    x2+=128

In [3]:
tiles

[(0, 0, 128, 128),
 (0, 128, 128, 256),
 (0, 256, 128, 384),
 (0, 384, 128, 512),
 (0, 512, 128, 640),
 (0, 640, 128, 768),
 (0, 768, 128, 896),
 (0, 896, 128, 1024),
 (0, 1024, 128, 1152),
 (0, 1152, 128, 1280),
 (0, 1280, 128, 1408),
 (0, 1408, 128, 1536),
 (0, 1536, 128, 1664),
 (0, 1664, 128, 1792),
 (0, 1792, 128, 1920),
 (0, 1920, 128, 2048),
 (0, 2048, 128, 2176),
 (0, 2176, 128, 2304),
 (0, 2304, 128, 2432),
 (0, 2432, 128, 2560),
 (0, 2560, 128, 2688),
 (0, 2688, 128, 2816),
 (0, 2816, 128, 2944),
 (0, 2944, 128, 3072),
 (0, 3072, 128, 3200),
 (0, 3200, 128, 3328),
 (0, 3328, 128, 3456),
 (0, 3456, 128, 3584),
 (0, 3584, 128, 3712),
 (0, 3712, 128, 3840),
 (0, 3840, 128, 3968),
 (0, 3968, 128, 4096),
 (0, 4096, 128, 4224),
 (0, 4224, 128, 4352),
 (0, 4352, 128, 4480),
 (0, 4480, 128, 4608),
 (0, 4608, 128, 4736),
 (0, 4736, 128, 4864),
 (0, 4864, 128, 4992),
 (0, 4992, 128, 5120),
 (0, 5120, 128, 5248),
 (0, 5248, 128, 5376),
 (0, 5376, 128, 5504),
 (0, 5504, 128, 5632),
 (0, 

In [15]:
x_tiles = []
for tile in tiles:
    tilearr = x_train_conv1[tile[0]:tile[2], tile[1]:tile[3],:]
    if tilearr.max() > 0:
        x_tiles.append(tilearr)

In [16]:
x_tiles1 = np.stack(x_tiles, axis = 0)

In [17]:
x_tiles1.shape

(6369, 128, 128, 12)

In [18]:
y_tiles = []
for tile in tiles:
    tilearr = y_train_conv[tile[0]:tile[2], tile[1]:tile[3],:]
    if tilearr.max() > 0:
        y_tiles.append(tilearr)

In [19]:
y_tiles1 = np.stack(y_tiles, axis = 0)

In [20]:
y_tiles1.shape

(6369, 128, 128, 1)

In [51]:
#splitting to train and test data / разделение на тренировочный и тестовый набор данных
x_tiles_train = x_tiles1[[samples[:5000]], :, :, :]
x_tiles_train = np.squeeze(x_tiles_train, axis=0)
y_tiles_train = y_tiles1[[samples[:5000]], :, :, :]
y_tiles_train = np.squeeze(y_tiles_train, axis=0)
y_tiles_train = keras.utils.to_categorical(y_tiles_train, num_classes=5)
x_tiles_test = x_tiles1[[samples[5000:]], :, :, :]
x_tiles_test = np.squeeze(x_tiles_test, axis=0)
y_tiles_test = y_tiles1[[samples[5000:]], :, :, :]
y_tiles_test = np.squeeze(y_tiles_test, axis=0)
y_tiles_test = keras.utils.to_categorical(y_tiles_test, num_classes=5)

In [58]:
#creating a model / создание модели
from keras import layers, models
inputs = layers.Input(shape = (128, 128, 12))

conv1 = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
conv1 = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = layers.Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = layers.Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = layers.Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = layers.Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = layers.Dropout(0.5)(conv4)
pool4 = layers.MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = layers.Conv2D(1024, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = layers.Conv2D(1024, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = layers.Dropout(0.5)(conv5)

up6 = layers.Conv2D(512, (2, 2), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(drop5))
merge6 = layers.concatenate([drop4,up6], axis = 3)
conv6 = layers.Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = layers.Conv2D(512, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = layers.Conv2D(256, (2, 2), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv6))
merge7 = layers.concatenate([conv3,up7], axis = 3)
conv7 = layers.Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = layers.Conv2D(256, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = layers.Conv2D(128, (2, 2), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv7))
merge8 = layers.concatenate([conv2,up8], axis = 3)
conv8 = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = layers.Conv2D(64, (2, 2), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv8))
merge9 = layers.concatenate([conv1,up9], axis = 3)
conv9 = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
#conv9 = layers.Conv2D(2, (3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
#conv10 = layers.Conv2D(1, 1, activation = 'softmax', padding = 'same')(conv9)
conv10 = layers.Conv2D(5, (1, 1), activation = 'softmax', padding = 'same')(conv9)

model = models.Model(inputs = inputs, outputs = conv10)

model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy', 'mse', 'AUC'])

In [60]:
#training a model / обучение модели
history = model.fit(x_tiles_train, y_tiles_train, batch_size = 8, epochs = 25, validation_data = (x_tiles_test, y_tiles_test))

Epoch 1/25
625/625 [==============================] - 163s 261ms/step - loss: 2.9677 - accuracy: 0.6944 - mse: 0.1086 - auc: 0.8693 - val_loss: 1.8307 - val_accuracy: 0.6558 - val_mse: 0.1144 - val_auc: 0.8765
Epoch 2/25
625/625 [==============================] - 163s 261ms/step - loss: 1.0361 - accuracy: 0.7525 - mse: 0.0780 - auc: 0.9307 - val_loss: 0.6951 - val_accuracy: 0.8000 - val_mse: 0.0602 - val_auc: 0.9563
Epoch 3/25
625/625 [==============================] - 163s 260ms/step - loss: 0.6310 - accuracy: 0.8023 - mse: 0.0588 - auc: 0.9588 - val_loss: 0.5631 - val_accuracy: 0.8319 - val_mse: 0.0513 - val_auc: 0.9681
Epoch 4/25
625/625 [==============================] - 163s 261ms/step - loss: 0.5227 - accuracy: 0.8238 - mse: 0.0515 - auc: 0.9675 - val_loss: 0.5182 - val_accuracy: 0.8150 - val_mse: 0.0533 - val_auc: 0.9660
Epoch 5/25
625/625 [==============================] - 163s 261ms/step - loss: 0.5007 - accuracy: 0.8236 - mse: 0.0509 - auc: 0.9689 - val_loss: 0.4561 - val_acc